# An un-normalized model example
A model of demand and supply, where the price is implicit determines. 

The model is first un-normalized: that is: $y = f(x,y)$ is changed to $0 = y - f(x,y)$

In [2]:
import pandas as pd
import numpy as np


from modelclass import model
import modelmanipulation as mp 

#model.modelflow_auto()
model.widescreen()

# Specify Model 

In [3]:
rdm = '''\
             demand = demand_ofset + demand_slope * price + demand_second * price**2
             supply = supply_ofset + supply_slope * price + supply_second * price**2
<endo=price> supply = demand  
'''

# rewrite the model as an un normalized model 0 = G(y,x)
The nodel is transformed. The goal of the newton solving is to make sure that `<variable>___RES` becomes very close to 0.  

In [4]:
fdm  = mp.un_normalize_simpel(rdm)
print(fdm)

FRML <> DEMAND___RES = ( DEMAND_OFSET + DEMAND_SLOPE * PRICE + DEMAND_SECOND * PRICE**2 ) - ( DEMAND ) $
FRML <> SUPPLY___RES = ( SUPPLY_OFSET + SUPPLY_SLOPE * PRICE + SUPPLY_SECOND * PRICE**2 ) - ( SUPPLY ) $
FRML <ENDO=PRICE> PRICE___RES = ( DEMAND ) - ( SUPPLY ) $


# Create a model instance 

In [5]:
mdm = model(fdm,modelname='A demand/supply model')

# Now we need some data, so we make a datataframe. 
it has three rows with different parameters 

In [6]:
grunddf = pd.DataFrame(0.0,index=[1,2,3],columns=['DEMAND_OFSET', 'DEMAND_SLOPE',
 'DEMAND_SECOND', 'SUPPLY_OFSET', 'SUPPLY_SLOPE', 'SUPPLY_SECOND', 'PRICE'])

# Stuff some values into the Dataframe
We start out with linear demand and supply curves

In [20]:
demandparam = 1,-0.5,0.0   # down sloping intersection 1, no second order 
supplyparam = 0.0,0.5,0.   # up sloping intersection 0, no second order 

grunddf.loc[:,:] = demandparam+supplyparam+(4.,)
grunddf.loc[:,'DEMAND_OFSET']=[1.0,0.9,0.8]

print(grunddf.T)

                 1    2    3
DEMAND_OFSET   1.0  0.9  0.8
DEMAND_SLOPE  -0.5 -0.5 -0.5
DEMAND_SECOND  0.0  0.0  0.0
SUPPLY_OFSET   0.0  0.0  0.0
SUPPLY_SLOPE   0.5  0.5  0.5
SUPPLY_SECOND  0.0  0.0  0.0
PRICE          4.0  4.0  4.0


# Run the model one experiment at a time 
As the model is linear, it will converge in one iteration

In [8]:
mdm.normalized


False

In [9]:
result0 = mdm(grunddf,max_iterations=30,silent=0,nonlin=2,forcenum=0,solver='newton_un_normalized',timeit=0)

Will start calculating: A demand/supply model
Prepare model for calculate derivatives for Newton solver
Model                           : A demand/supply model
Number of endogeneus variables  : 3
Number of derivatives           : 6
A demand/supply model Derivatives  no lags and leads calculated  
Iteration  0 Sum of distances        3.000000
Iteration  1 Sum of distances        0.000000
1 Solved in 1 iterations
Iteration  0 Sum of distances        3.100000
Iteration  1 Sum of distances        0.000000
2 Solved in 1 iterations
Iteration  0 Sum of distances        3.200000
Iteration  1 Sum of distances        0.000000
3 Solved in 1 iterations
A demand/supply model solved  


In [10]:
result0.PRICE

1    1.0
2    0.9
3    0.8
Name: PRICE, dtype: float64

In [11]:
mdm.newton_diff.get_diff_df_1per()

A demand/supply model Derivatives  no lags and leads calculated  


{1:               DEMAND___RES  PRICE___RES  SUPPLY___RES
 DEMAND___RES          -1.0         -0.5           0.0
 PRICE___RES            1.0          0.0          -1.0
 SUPPLY___RES           0.0          0.5          -1.0,
 2:               DEMAND___RES  PRICE___RES  SUPPLY___RES
 DEMAND___RES          -1.0         -0.5           0.0
 PRICE___RES            1.0          0.0          -1.0
 SUPPLY___RES           0.0          0.5          -1.0,
 3:               DEMAND___RES  PRICE___RES  SUPPLY___RES
 DEMAND___RES          -1.0         -0.5           0.0
 PRICE___RES            1.0          0.0          -1.0
 SUPPLY___RES           0.0          0.5          -1.0}

In [16]:
mat = mdm.newton_diff.get_diff_df_1per()[1]
mat

A demand/supply model Derivatives  no lags and leads calculated  


,DEMAND___RES,PRICE___RES,SUPPLY___RES
DEMAND___RES,-1.0,-0.5,0.0
PRICE___RES,1.0,0.0,-1.0
SUPPLY___RES,0.0,0.5,-1.0


In [18]:
mat_inv = pd.DataFrame(np.linalg.inv(mat.values), mat.columns, mat.index)
mat_inv

,DEMAND___RES,PRICE___RES,SUPPLY___RES
DEMAND___RES,-0.5,0.5,-0.5
PRICE___RES,-1.0,-1.0,1.0
SUPPLY___RES,-0.5,-0.5,-0.5


In [19]:
mat @ mat_inv

,DEMAND___RES,PRICE___RES,SUPPLY___RES
DEMAND___RES,1.0,0.0,0.0
PRICE___RES,0.0,1.0,0.0
SUPPLY___RES,0.0,0.0,1.0


# Now make the demand nonlinear 
By setting DEMAND_SECOND to a non-zero value  

Now solving requires more iterations and updating of the Jacobi matrix

In [10]:
eksdf = model.update_from_list(grunddf,'DEMAND_SECOND = 0.2') 
print(eksdf)

   DEMAND_OFSET  DEMAND_SLOPE  DEMAND_SECOND  SUPPLY_OFSET  SUPPLY_SLOPE  \
1           1.0          -0.5            0.2           0.0           0.5   
2           0.9          -0.5            0.2           0.0           0.5   
3           0.8          -0.5            0.2           0.0           0.5   

   SUPPLY_SECOND  PRICE  
1            0.0    4.0  
2            0.0    4.0  
3            0.0    4.0  


In [11]:
result1 = mdm(eksdf,max_iterations=30,silent=0,nonlin=2,forcenum=0,newton_reset=1)
result1.PRICE

Will start calculating: A demand/supply model
A demand/supply model Derivatives  no lags and leads calculated  
Iteration  0 Sum of distances        4.200000
Iteration  1 Sum of distances        0.022222
Iteration  2 Sum of distances        0.005213
Updating solver, iteration 2
A demand/supply model Derivatives  no lags and leads calculated  
Iteration  3 Sum of distances        0.000027
1 Solved in 3 iterations
Iteration  0 Sum of distances        4.100000
Iteration  1 Sum of distances        0.022991
Iteration  2 Sum of distances        0.003552
Updating solver, iteration 2
A demand/supply model Derivatives  no lags and leads calculated  
Iteration  3 Sum of distances        0.000009
2 Solved in 3 iterations
Iteration  0 Sum of distances        4.000000
Iteration  1 Sum of distances        0.000000
3 Solved in 1 iterations
A demand/supply model solved  


1    3.618094
2    3.822893
3    4.000000
Name: PRICE, dtype: float64

# Look at the results

In [12]:
mdm.price 

Exogeneous : PRICE:   
 Values : 
             1         2    3
Base  1.000000  0.900000  0.8
Last  3.618094  3.822893  4.0
Diff  2.618094  2.922893  3.2

# Inspect the differential coefficients


In [13]:
mdm.newton_diff.show_diff_latex()

A demand/supply model Derivatives  no lags and leads calculated  


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

|                                                        |    1 |    2 |    3 |
|:-------------------------------------------------------|-----:|-----:|-----:|
| $\frac{\partial SUPPLY\_\_\_RES_t}{\partial PRICE_t}$  |  0.5 |  0.5 |  0.5 |
| $\frac{\partial SUPPLY\_\_\_RES_t}{\partial SUPPLY_t}$ | -1   | -1   | -1   |

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

|                                                       |   1 |   2 |   3 |
|:------------------------------------------------------|----:|----:|----:|
| $\frac{\partial PRICE\_\_\_RES_t}{\partial DEMAND_t}$ |   1 |   1 |   1 |
| $\frac{\partial PRICE\_\_\_RES_t}{\partial SUPPLY_t}$ |  -1 |  -1 |  -1 |

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

|                                                        |         1 |        2 |    3 |
|:-------------------------------------------------------|----------:|---------:|-----:|
| $\frac{\partial DEMAND\_\_\_RES_t}{\partial DEMAND_t}$ | -1        | -1       | -1   |
| $\frac{\partial DEMAND\_\_\_RES_t}{\partial PRICE_t}$  |  0.947237 |  1.02916 |  1.1 |

In [16]:
mdm.newton_diff.get_diff_df_1per()

A demand/supply model Derivatives  no lags and leads calculated  


{1:               DEMAND___RES  PRICE___RES  SUPPLY___RES
 DEMAND___RES          -1.0     0.947237           0.0
 PRICE___RES            1.0     0.000000          -1.0
 SUPPLY___RES           0.0     0.500000          -1.0,
 2:               DEMAND___RES  PRICE___RES  SUPPLY___RES
 DEMAND___RES          -1.0     1.029157           0.0
 PRICE___RES            1.0     0.000000          -1.0
 SUPPLY___RES           0.0     0.500000          -1.0,
 3:               DEMAND___RES  PRICE___RES  SUPPLY___RES
 DEMAND___RES          -1.0          1.1           0.0
 PRICE___RES            1.0          0.0          -1.0
 SUPPLY___RES           0.0          0.5          -1.0}